<b> ZKOUSKA prikazu podle webu </b>

In [7]:
import psycopg2
import pandas as pd

print('Manual connecting to the PostgreSQL database...\n\n\n')
conn = psycopg2.connect(
    host="postgres",
    database="dvdrental",
    user="postgres",
    password="example")

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

# create a cursor
cur = conn.cursor()

#LIST TABULEK BASE TABLE
tab_info = create_pandas_table("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE' AND table_schema NOT IN ('pg_catalog', 'information_schema');")

print('Tabulky v databazi:\n')
print(tab_info)

#LIST TABULEK VIEWs
view_info = create_pandas_table("SELECT table_name FROM information_schema.tables WHERE table_type = 'VIEW' AND table_schema NOT IN ('pg_catalog', 'information_schema') AND table_name !~ '^pg_';")
print('\n\n\nViews v aktualni databazi: \n')
print(view_info)

#LIST USERS
list_users = create_pandas_table("SELECT usename FROM pg_user;")
print('\n\n\nUzivatele v aktualni databazi: \n')
print(list_users)

#RADKY v databazi
colum_info = create_pandas_table("SELECT column_name FROM information_schema.columns WHERE table_name = 'store';")

print('\n\n\nINFO O tabulce: *JMENOTABULKY* \n')
print(colum_info)

#Detailni info o radcich
detail_info = create_pandas_table("SELECT ordinal_position, column_name, data_type, column_default, is_nullable, character_maximum_length, numeric_precision FROM information_schema.columns WHERE table_name = 'store' ORDER BY ordinal_position;")

print('\n\n Detailní info o tabulce : *JMENOTABULKY* \n')
print(detail_info)

#LIST TRIGGERS#
trigger_info = create_pandas_table("SELECT DISTINCT trigger_name FROM information_schema.triggers WHERE trigger_schema NOT IN ('pg_catalog', 'information_schema');")

print('\n\n List of triggers : \n')
print(trigger_info)

#Detailni LIST TRIGGERS (pro vsechno v databazi)#
#Detail_trigger_info = create_pandas_table("SELECT * FROM information_schema.triggers WHERE trigger_schema NOT IN ('pg_catalog', 'information_schema');")
#print('\n\n Detailni List of triggrs : \n')
#print(Detail_trigger_info)

# close the communication with the PostgreSQL
cur.close()
conn.close()

Manual connecting to the PostgreSQL database...



Tabulky v databazi:

       table_name
0           actor
1           store
2         address
3        category
4            city
5         country
6        customer
7      film_actor
8   film_category
9       inventory
10       language
11         rental
12          staff
13        payment
14           film



Views v aktualni databazi: 

                   table_name
0                  actor_info
1               customer_list
2                   film_list
3  nicer_but_slower_film_list
4      sales_by_film_category
5              sales_by_store
6                  staff_list



Uzivatele v aktualni databazi: 

    usename
0  postgres



INFO O tabulce: *JMENOTABULKY* 

        column_name
0          store_id
1  manager_staff_id
2        address_id
3       last_update


 Detailní info o tabulce : *JMENOTABULKY* 

   ordinal_position       column_name                    data_type  \
0                 1          store_id                   

<b>pomocí SQL prikazu dotaz jak vypada schema a tabulka</b>

In [89]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="postgres",
    database="postgres",
    user="postgres",
    password="example")

#kurzor na cteni/zapis z databaze pomoci pythonu
cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table


#SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';

schemata = create_pandas_table("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';")

print('Schemata v databazi:\n')
print(schemata)

table_schema = create_pandas_table("SELECT * FROM information_schema.columns WHERE TABLE_NAME = 'uzivatel';")
 

print('\n\nSchemata v tabulce:\n')
print(table_schema)

shZK = create_pandas_table("SELECT schemaname FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';")
print('\n\n SCHEMATA?:\n')
print(shZK)


cur.close()
conn.close()

Schemata v databazi:

  schemaname   tablename tableowner tablespace  hasindexes  hasrules  \
0     public       users   postgres       None        True     False   
1     public  usergroups   postgres       None        True     False   
2     public      groups   postgres       None        True     False   
3     public    uzivatel   postgres       None        True     False   

   hastriggers  rowsecurity  
0         True        False  
1         True        False  
2         True        False  
3        False        False  


Schemata v tabulce:

  table_catalog table_schema table_name     column_name  ordinal_position  \
0      postgres       public   uzivatel    uzivatele_id                 1   
1      postgres       public   uzivatel           jmeno                 2   
2      postgres       public   uzivatel        prijmeni                 3   
3      postgres       public   uzivatel  datum_narozeni                 4   
4      postgres       public   uzivatel    pocet_clanku    

In [26]:
pip install flask

     |████████████████████████████████| 94 kB 2.2 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


<h1><b> ZÍSKÁNÍ struktury z databaze BETA-2.1.0 </b></h1>

In [32]:
import psycopg2
import flask

#------------Pripojeni-k-databazi---------------------
conn = psycopg2.connect(
    host="postgres",
    database="postgres", #TESTOVACI DATABAZE: postgres,dvdrental
    user="postgres",
    password="example")

#------------Vytvoreni-kurzoru-------------------
db_cursor = conn.cursor()

#---------------------Najiti-vsech-schemat-v-databazi-------------------------------------
def GetSchema():
    p = "SELECT schemaname FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';" #ignorovani schemat catalog a information
    
    db_cursor.execute(p)
    list_schemat = db_cursor.fetchall()
    
    list_schemat = list(dict.fromkeys(list_schemat)) #odtraneni duplicitnich schemat v listu a vytvoreni noveho kde je kazde schema jednou !
    
    return(list_schemat)

#------------------------Ziskani-Views-z-databaze--------------------------------   
def GetViewsList():
    v = "SELECT table_name FROM information_schema.views WHERE table_schema NOT IN ('pg_catalog', 'information_schema');"
    
    db_cursor.execute(v)
    list_views = db_cursor.fetchall()
    #print(list_views)
    return(list_views)
    #for view in list_views:
    #    print(view[0])
    #return(list_views)

#------------------Najiti-tabulek-v-kazdem-schematu-v-databazi--------------------------------------
def GetTableList(schema,view_li):
    s = ""
    s += "SELECT"
    s += " table_name"
    s += " FROM information_schema.tables"
    s += " WHERE"
    s += " ("
    s += " table_schema = '" + schema + "'"
    s += " );"
    #s += " ORDER BY table_schema, table_name;"

    # Retrieve all the rows from the cursor
    db_cursor.execute(s)
    list_tables = db_cursor.fetchall()

    # Print the names of the tables
#----Odstraneni Views prvku z tabulek----
    new_list_tables=[]
    for tabl in list_tables:
        x=0
        for view in view_li:
            #print(view[0])
            if tabl[0]==view[0]:                
                x=1
                list_tables
                break
        if x==0:
            new_list_tables.append(tabl)
        #print(tabl[0])
    #    list_tables.remove(view_li)
    #print("NOVY LIST bez Views: " + str(new_list_tables))
    return(new_list_tables)
    #for t_name_table in list_tables:
        #print(t_name_table)
     #   return(t_name_table) # POUZE jmeno tabulky

#----------------------Popis-radku-v-tabulkach-------------------------------------
def GetColInfo(radek):
    l = "SELECT"
    l += " table_name, ordinal_position, column_name, data_type, column_default, is_nullable, character_maximum_length, numeric_precision"
    l += " FROM information_schema.columns"
    l += " WHERE TABLE_NAME = '" + radek + "'"
    l += " ORDER BY ordinal_position;"
    
    db_cursor.execute(l)
    list_colums = db_cursor.fetchall()
    
    #print(list_colums)
    return(list_colums)
    #for nam in list_colums:
    #    print(nam)

#--------------------Vypis-klicu-v-tabulkach----------------------------------------
def GetKeyList(tab):
    k = "SELECT"       #AND tc.constraint_name = 'usergroups_group_id_fkey';"
    k += " tc.constraint_name, tc.constraint_type, kcu.column_name, tc.is_deferrable, tc.initially_deferred,"
    k += " rc.match_option AS match_type, rc.update_rule AS on_update, rc.delete_rule AS on_delete,"
    k += " ccu.table_name AS references_table, ccu.column_name AS references_field"
    k += " FROM information_schema.table_constraints tc"
    k += " LEFT JOIN information_schema.key_column_usage kcu ON tc.constraint_catalog = kcu.constraint_catalog"
    k += " AND tc.constraint_schema = kcu.constraint_schema AND tc.constraint_name = kcu.constraint_name"
    k += " LEFT JOIN information_schema.referential_constraints rc ON tc.constraint_catalog = rc.constraint_catalog"
    k += " AND tc.constraint_schema = rc.constraint_schema AND tc.constraint_name = rc.constraint_name"
    k += " LEFT JOIN information_schema.constraint_column_usage ccu ON rc.unique_constraint_catalog = ccu.constraint_catalog"
    k += " AND rc.unique_constraint_schema = ccu.constraint_schema AND rc.unique_constraint_name = ccu.constraint_name"
    k += " WHERE tc.table_name = '" + tab + "';"
    
    db_cursor.execute(k)
    list_keys = db_cursor.fetchall()
    ####--zbaven-se-CHECK-klicu,-ktere-zpusobuji-chybu--####
    #print("puvodni klice: " + str(list_keys)+" \n")
    # POUZITO try: except Exception: pass
    
    #print(list_keys)
    return(list_keys)
    #for klic in list_keys:
    #    print(klic)
    

#---------------------Spusteni-hlednai-v-databazi-a-ukladani-do-TEXTAKU-----------------------------------       
f = open("SQL_STRUCT.txt","w+")

TestSchema = GetSchema()
ViewList = GetViewsList()
for sch in TestSchema:
    #print(sch[0])
    schema = sch[0]
   # f.write("CREATE SCHEMA "+sch[0]+"\n")
    seznam_tabulek = GetTableList(schema,GetViewsList())
    print(seznam_tabulek)
    for tabulka in seznam_tabulek:
        #print("Jmeno tabulky: " + str(tabulka[0]))
        f.write("class "+sch[0]+"." + tabulka[0]+"(BaseModel):\n")
        f.write("\t__tablename__ = '" + tabulka[0]+"' \n")
        Column_Info = GetColInfo(tabulka[0])
        #GetColInfo(tabulka[0])
        
        schem_info_list =[]
        
        for column in Column_Info:
            print("\nRADEK : " + str(column)+"\n")
           
            f.write("\n\t"+column[2]+" = "+"Column(")
            
            schem_info_list.append(column[2])
            
            if column[3]=="timestamp without time zone":    # Prevod typu "timestamp without time zone"
                f.write("DateTime")
                schem_info_list.append("DateTime")
            elif column[3]=="character varying":
                f.write("String")                           # MOŽNÁ ?????? nebo nechat "character varying" FUNGUJE V PYTHONU ????
                schem_info_list.append("str")
                if column[6]!=None:
                    f.write("("+ str(column[6]) +")")
            elif column[3]=="integer" or "smallint" or "bigint": # Zachycení vsech intů (BigInt,SmallInt,int)
                f.write("Integer")
                schem_info_list.append("int")
            else:
                f.write(column[3])
                schem_info_list.append(column[3])
            
            if column[4]!=None:
                f.write(", Sequence(" + column[4] +")")
            seznam_keys = GetKeyList(tabulka[0])
        
        #print("SEZNAM KLICU " + str(seznam_keys))
        #pocitadlo chybovych CONSTRAINT
            poci=0
            for klic in seznam_keys:
                if klic[1]=="CHECK":
                    poci=poci+1
            i=0
            for klic in seznam_keys:
                print(klic)
                i=i+1
                pas=0
                print("Pocet spravnych klicu: "+str(len(seznam_keys)-poci))
                try: #z duvodu Non_type u CHECK constraint
                    if klic[1]=="PRIMARY KEY" and klic[2]==column[2]:
                        f.write(", primary_key=True")
                    else:
                        pass
                    if klic[1]=="FOREIGN KEY" and klic[2]==column[2]:
                        f.write(", ForeignKey(" )                           #   + "__TEST__"       # Doplnit CO ZDE BUDE ???????? mozna nazev klice ????
                        f.write("'" + klic[8]+"."+klic[9]+ "'")
                        if klic[6]!=None:
                            f.write(", onupdate = '"+ klic[6] +"'")
                        if klic[7]!=None:
                            f.write(", ondelete = '"+ klic[7] +"'")
                        else:
                            pass
                        f.write(")")
                    #f.write("\t\tCONSTRAINT " + klic[0] + " " + klic[1] + " (" + klic[2] + ")")
                    
                except Exception:
                    pas=1 # abychom vedeli kdy jsme na spatnem constraint prvku a mohli vynechat carku na konci radku
                    pass
            f.write(")")
        '''
            if column[6]!=None:
                f.write(","+str(column[6])+")")
            if column[5]=="NO":
                f.write(", NOT NULL")
            if column[4]!=None:
                f.write(", DEFAULT "+ str(column[4]))
            #if i!= len(Column_Info):
            f.write(")\n")
            #else:
            #    f.write("\n\t)\n")
        '''   
            #print(column[2])
            #print(len(Column_Info))
            
        print("\n") #pouze pro prehledne hledani
        #GetKeyList(tabulka[0])
        seznam_keys = GetKeyList(tabulka[0])
        
        #print("SEZNAM KLICU " + str(seznam_keys))
        #pocitadlo chybovych CONSTRAINT
    #######   poci=0
    #######   for klic in seznam_keys:
    #######       if klic[1]=="CHECK":
    #######           poci=poci+1
    #######   i=0
    #######   for klic in seznam_keys:
    #######       print(klic)
    #######       i=i+1
    #######       pas=0
    #######       print("Pocet spravnych klicu: "+str(len(seznam_keys)-poci))
    #######       try: #z duvodu Non_type u CHECK constraint
    #######           if klic[1]=="PRIMARY KEY":
    #######               f.write(" primary_key=True")
    #######           else:
    #######               pass
    #######           #f.write("\t\tCONSTRAINT " + klic[0] + " " + klic[1] + " (" + klic[2] + ")")
    #######       except Exception:
    #######           pas=1 # abychom vedeli kdy jsme na spatnem constraint prvku a mohli vynechat carku na konci radku
    #######           pass
    #######   #    
        #    if klic[8]!=None:
        #        f.write("\n\t\t\tREFERENCES " + sch[0] + "." + klic[8] + " (" + klic[9] +") MATCH SIMPLE")
        #    if klic[6]!=None:
        #        f.write("\n\t\t\tON UPDATE " + klic[6])
        #    if klic[7]!=None:
        #        f.write("\n\t\t\tON DELETE " + klic[7])
        #    
        #    if i!= (len(seznam_keys)-poci) and pas!=1:
        #        f.write(",\n")
        #    elif i==(len(seznam_keys)-poci):
        #        f.write("\n\t)\n\n")
        print("\n") #pouze pro prehledne hledani
        f.write("\n\n")
        
        #### VYTVORENI SCHEMAT:
        
        f.write("class "+ tabulka[0] +"Create(BaseModel):\n")
        po=1
        for schmta in schem_info_list:
            if (po % 2) != 0:
                f.write("\t" + schmta +": ")
            else:
                f.write(schmta + "\n")
            po=po+1
        f.write("\n")
        
        f.write("class "+ tabulka[0] +"Get(BaseModel):\n")
        po=1
        for schmta in schem_info_list:
            if (po % 2) != 0:
                f.write("\t" + schmta +": ")
            else:
                f.write(schmta + "\n")
            po=po+1
        f.write("\tclass Config:\n\t\torm_mode = True\n\tpass\n")
        f.write("\n")
        
        f.write("class "+ tabulka[0] +"Update(BaseModel):\n")
        po=1
        for schmta in schem_info_list:
            if (po % 2) != 0:
                f.write("\t" + schmta +": ")
            else:
                f.write(schmta + "\n")
            po=po+1
        f.write("\n")
        
        f.write("class "+ tabulka[0] +"Delete(BaseModel):\n")
        po=1
        for schmta in schem_info_list:
            if (po % 2) != 0:
                f.write("\t" + schmta +": ")
            else:
                f.write(schmta + "\n")
            po=po+1
        f.write("\n\n")
        #l= "SELECT tc.constraint_name, tc.constraint_type, kcu.column_name, tc.is_deferrable, tc.initially_deferred, rc.match_option AS match_type, rc.update_rule AS on_update, rc.delete_rule AS on_delete, ccu.table_name AS references_table, ccu.column_name AS references_field FROM information_schema.table_constraints tc LEFT JOIN information_schema.key_column_usage kcu ON tc.constraint_catalog = kcu.constraint_catalog AND tc.constraint_schema = kcu.constraint_schema AND tc.constraint_name = kcu.constraint_name LEFT JOIN information_schema.referential_constraints rc ON tc.constraint_catalog = rc.constraint_catalog AND tc.constraint_schema = rc.constraint_schema AND tc.constraint_name = rc.constraint_name LEFT JOIN information_schema.constraint_column_usage ccu ON rc.unique_constraint_catalog = ccu.constraint_catalog AND rc.unique_constraint_schema = ccu.constraint_schema AND rc.unique_constraint_name = ccu.constraint_name WHERE tc.table_name = 'usergroups';" #AND tc.constraint_name = 'usergroups_group_id_fkey';"

        
#schema = "public"
#seznam_tabulek = GetTableList(schema)
#for tabulka in seznam_tabulek:
#    GetColInfo(tabulka[0])
    #print(s[0])
#print(seznam_tabulek)
#rad = "uzivatel"
#GetColInfo(rad)

                                                                           #DODELAT: CREATE OR REPLACE VIEW !!!!! pripadne odstranit z tabulky at se nevraci
                                                                           #         mozna poskladat tabulky podle PORADI !!!! hotovo pomoci ORDER BY ordinal_position

#---------!! UKONCENI SPOJENI !!-----------
db_cursor.close()
conn.close()

f.close()

[('users',), ('usergroups',), ('groups',), ('uzivatel',)]

RADEK : ('users', 1, 'id', 'bigint', None, 'NO', None, 64)

('users_pkey', 'PRIMARY KEY', 'id', 'NO', 'NO', None, None, None, None, None)
Pocet spravnych klicu: 1
('2200_16856_1_not_null', 'CHECK', None, 'NO', 'NO', None, None, None, None, None)
Pocet spravnych klicu: 1

RADEK : ('users', 2, 'name', 'character varying', None, 'YES', None, None)

('users_pkey', 'PRIMARY KEY', 'id', 'NO', 'NO', None, None, None, None, None)
Pocet spravnych klicu: 1
('2200_16856_1_not_null', 'CHECK', None, 'NO', 'NO', None, None, None, None, None)
Pocet spravnych klicu: 1





RADEK : ('usergroups', 1, 'id', 'bigint', None, 'NO', None, 64)

('usergroups_pkey', 'PRIMARY KEY', 'id', 'NO', 'NO', None, None, None, None, None)
Pocet spravnych klicu: 3
('usergroups_user_id_fkey', 'FOREIGN KEY', 'user_id', 'NO', 'NO', 'NONE', 'NO ACTION', 'NO ACTION', 'users', 'id')
Pocet spravnych klicu: 3
('usergroups_group_id_fkey', 'FOREIGN KEY', 'group_id', 'NO', 'N